In [1]:
%matplotlib inline
import pyspeckit
import numpy as np
from astropy import units as u
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy.convolution import convolve, Box1DKernel
from c_normalize import c_normalize

# Python-based Automation of Equivalent Width Measurement
Utilizing pyspeckit (https://pyspeckit.readthedocs.io/en/latest/) and a python port of Joleen Carlberg's (STScI) Continuum Normalization algorithm, this notebook seeks to set up and test an automated equivalent width measurement algorithm for a number of spectrum for which we have hand measurements. The expectations for this method are that 95% of the measurements are within 5% of our by-hand measurements.

### An Example
Before diving straight into scripting, it's useful to layout an individual example of the equivalent width measurement process.

In [ ]:
s_hdu = fits.open("Data/ew_known/tame_inputs/col110_1134red_oned_25jan14_wavsoln.fits")
fig = plt.figure(figsize=(15,10))

line_cen = 5307.36
width = 10
lim_l = line_cen - width
lim_r = line_cen + width


s_data = s_hdu[1].data
s_flux = s_data['FLUX']
smoothed_flux = convolve(s_flux, Box1DKernel(5))
s_flux = smoothed_flux
s_wav = s_data['WAVEL']

wav_mask = (s_wav > lim_l) & (s_wav < lim_r)
s_flux = s_flux[wav_mask]
s_wav = s_wav[wav_mask]
yfit, norm, pts = c_normalize(s_flux, s_wav, median_replace=False, cheby=True, low_cut = 0.9)

sp = pyspeckit.Spectrum(data=norm, xarr=s_wav * u.AA)
sp.plotter(figure = fig, xmin=lim_l, xmax=lim_r, ymin=0, ymax=1.2)

#Get around pyspeckits reliance on a generated baseline and just generate basespec as a ones array (continuum is always one in normalized case)
sp.baseline.basespec = np.ones(len(s_wav))
sp.specfit(fittype='gaussian', guesses = [0.1, 5307.36, 0.1])
EQW = sp.specfit.EQW(plot=True, continuum_as_baseline=True)
print("EQW:", EQW*1000*u.mA)
plt.axhline(1, color = 'orange')
#sp.plotter.savefig('basic_plot_example_withfit.png')